<a href="https://colab.research.google.com/github/Ahmed-ali-fcai-811/machine-learning/blob/main/Assignment2_optimaizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from keras.datasets import mnist
import numpy as np
from sklearn.model_selection import train_test_split

In [30]:
(x , y) , (x_test , y_test) = mnist.load_data()

In [31]:
# 0 - 1 mask 
train_data_msk  = np.where((y==1) | (y == 0))
test_data_msk = np.where( (y_test == 1) | (y_test == 0))

In [32]:
x = x[train_data_msk]
y = y[train_data_msk]

In [33]:
x_test = x_test[test_data_msk]
y_test = y_test[test_data_msk]

In [34]:
x = x.reshape(x.shape[0] ,-1 )

In [35]:
x.shape

(12665, 784)

In [36]:
x_test = x_test.reshape(x_test.shape[0] ,-1 )

In [37]:
y_test.shape

(2115,)

In [38]:
x_test = (x_test - np.mean(x)) / np.std(x)
x = (x - np.mean(x)) / np.std(x)

In [40]:
def accuarcy(y_pred , y):
  acc = 0 
  for i in range (y_pred.shape[0]):
    acc+=(y_pred[i] == y[i])

  return( acc/y_pred.shape[0])*100

In [41]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

In [39]:
class LogisticRegression:
  def __init__(self , eta , n_iters ,x,y):
    self.eta = eta 
    self.n_iters = n_iters
    self.w = np.zeros((x.shape[1] , ))
    self.b = 0 



  def segmoid(self , z):
    return 1/(1+np.exp(-z))

  def predict(self , x):
    output = np.dot(self.w.T , x.T )+self.b
    pred = self.segmoid(output)
    pred = np.where(pred >= 0.5 ,1  , 0)
    return pred


  def fit(self , x , y):
    n_samples , n_features = x.shape
    
    for i in range(self.n_iters):
      y_pred = self.predict(x)
      c = (y_pred - y)
      dj_dw = (1/n_samples) * np.dot(x.T , c)
      dj_db = (1/n_samples) * np.sum(c)

      self.w -= self.eta * dj_dw
      self.b -= self.eta * dj_db

    return self.w , self.b


In [42]:
class L1_regularization:
  def __init__(self , eta , n_iters, lamda):
    self.eta = eta 
    self.n_iters = n_iters
    self.lamda =lamda

  def segmoid(self , z):
    return 1/(1+np.exp(-z))

  def predict(self , x):
    output = np.dot(self.w.T , x.T )+self.b
    pred = self.segmoid(output)
    pred = np.where(pred >= 0.5 ,1  , 0)
    return pred


  def fit(self , x , y):
    n_samples , n_features = x.shape
    self.w = np.zeros((n_features , ))
    self.b = 0 

    for i in range(self.n_iters):
      y_pred = self.predict(x)
      c = (y_pred - y)
      penalty = self.lamda * np.sign(self.w)
      dj_dw = (1/n_samples) * np.dot(x.T , c)+penalty
      dj_db = (1/n_samples) * np.sum(c)

      self.w -= self.eta * dj_dw
      self.b -= self.eta * dj_db

    return self.w , self.b


In [52]:
def mini_batch_GD(x,y,num_batches ,x_val,y_val):
  model =LogisticRegression(0.01,1,x,y)
  batch_size =len(x)//num_batches
  for i in  range(num_batches):
     X_batch = x[i * batch_size : (i + 1) * batch_size]
     y_batch = y[i * batch_size : (i + 1) * batch_size]
     model.fit(X_batch,y_batch)

  
  y_pred=model.predict(x_val)

  return accuarcy(y_pred,y_val)



In [62]:
class RMS_Prob_optimizer:
  def __init__(self , eta , n_iters ,beta=0.99,epsilon=1e-8):
    self.eta = eta 
    self.n_iters = n_iters
    self.beta =beta
    self.epsilon = epsilon

  def segmoid(self , z):
    return 1/(1+np.exp(-z))

  def predict(self , x):
    output = np.dot(self.w.T , x.T )+self.b
    pred = self.segmoid(output)
    pred = np.where(pred >= 0.5 ,1  , 0)
    return pred


  def fit(self , x , y):
    n_samples , n_features = x.shape
    self.w = np.zeros((n_features , ))
    self.b = 0 
    self.sdw=0
    self.sdb=0

    for i in range(self.n_iters):
      y_pred = self.predict(x)
      c = (y_pred - y)
      dj_dw = (1/n_samples) * np.dot(x.T , c)
      dj_db = (1/n_samples) * np.sum(c)

      self.sdw=self.sdw*self.beta +(1-self.beta)*dj_dw**2
      self.sdb=self.sdb*self.beta +(1-self.beta)*dj_db**2

      self.w -= self.eta * (dj_dw/np.sqrt(self.sdw)+self.epsilon)
      self.b -= self.eta * (dj_db/np.sqrt(self.sdb)+self.epsilon)

    return self.w , self.b


In [68]:
class Adam_optimizer:
  def __init__(self , eta , n_iters ,beta=0.99,epsilon=1e-8):
    self.eta = eta 
    self.n_iters = n_iters
    self.beta =beta
    self.epsilon = epsilon

  def segmoid(self , z):
    return 1/(1+np.exp(-z))

  def predict(self , x):
    output = np.dot(self.w.T , x.T )+self.b
    pred = self.segmoid(output)
    pred = np.where(pred >= 0.5 ,1  , 0)
    return pred


  def fit(self , x , y):
    n_samples , n_features = x.shape
    self.w = np.zeros((n_features , ))
    self.b = 0 
    self.sdw=0
    self.sdb=0
    self.vdw=0
    self.vdb=0

    for i in range(self.n_iters):
      y_pred = self.predict(x)
      c = (y_pred - y)
      dj_dw = (1/n_samples) * np.dot(x.T , c)
      dj_db = (1/n_samples) * np.sum(c)

      self.sdw=self.sdw*self.beta +(1-self.beta)*dj_dw**2
      self.sdb=self.sdb*self.beta +(1-self.beta)*dj_db**2
      
      
      self.vdw=self.vdw*self.beta +(1-self.beta)*dj_dw
      self.vdb=self.vdb*self.beta +(1-self.beta)*dj_db

      self.w -= self.eta * (self.vdw/np.sqrt(self.sdw)+self.epsilon)
      self.b -= self.eta * (self.vdb/np.sqrt(self.sdb)+self.epsilon)

    return self.w , self.b


In [51]:
#use  0.1,0.5 for lampda
#use 5batch,10